<a href="https://colab.research.google.com/github/gmihaila/machine_learning_toolbox/blob/master/pytorch_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SImple NN

1 hiddent layer NN

In [0]:
import torch

n_input, n_hidden, n_output = 5, 3, 1

## initialize tensor for inputs, and outputs 
X = torch.randn((1, n_input))
y = torch.rand((1,n_output)) 


print(x.size())
print(y.size())
print()

## initialize tensor variables for weights 
w1 = torch.rand((n_input, n_hidden))
w2 = torch.rand((n_hidden, n_output))

print(w1.size())
print(w2.size())
print()

## initialize tensor variables for bias terms 
b1 = torch.rand((1,n_hidden))
b2 = torch.rand((1,n_output))

print(b1.size())
print(b2.size())
print()



torch.Size([1, 5])
torch.Size([1, 1])

torch.Size([5, 3])
torch.Size([3, 1])

torch.Size([1, 3])
torch.Size([1, 1])


1. Forward Propagation
2. Loss computation
3. Backpropagation
4. Updating the parameters

In [0]:
## sigmoid activation function using pytorch
def sigmoid_activation(z):
  return 1 / (1 + torch.exp(-z))

## activation of hidden layer 
z1 = torch.mm(X,w1) + b1
a1 = sigmoid_activation(z1)

print(z1)
print(a1)
print()

## activation (output) of final layer 
z2 = torch.mm(a1, w2) + b2
a2 = output = sigmoid_activation(z2)

print(z2)
print(output)
print()

loss = y - output

print(loss)

tensor([[-0.0190, -1.0797, -0.6849]])
tensor([[0.4952, 0.2536, 0.3352]])

tensor([[0.8171]])
tensor([[0.6936]])

tensor([[0.0193]])


### Backprop

* loss gets multiplied by weights to penalize more of the bad weights
* some weights contirbute more to the output. If the error is large, their loss will be more

In [0]:
## function to calculate the derivative of activation
def sigmoid_delta(x):
  return x * (1 - x)

## compute derivative of error terms
delta_output = sigmoid_delta(output)
delta_hidden = sigmoid_delta(a1)

print(delta_output)
print(delta_hidden)
print()


## backpass the changes to previous layers 
d_output = loss * delta_output
loss_h = torch.mm(d_output, w2.t())
d_hidden = loss_h * delta_hidden

print(d_output)
print(loss_h)
print(d_hidden)

tensor([[0.2125]])
tensor([[0.2500, 0.1893, 0.2228]])

tensor([[0.0041]])
tensor([[0.0022, 0.0008, 0.0002]])
tensor([[5.4456e-04, 1.5298e-04, 3.9821e-05]])


### Update Parameters

In [0]:
learning_rate = 0.1

w2 += torch.mm(a1.t(), d_output) * learning_rate
w1 += torch.mm(X.t(), d_hidden) * learning_rate


print(w2)
print(w1)
print()

b1 += d_output.sum() * learning_rate
b2 += d_hidden.sum() * learning_rate

print(b1)
print(b2)

tensor([[0.5323],
        [0.1976],
        [0.0441]])
tensor([[0.4660, 0.4263, 0.1392],
        [0.5357, 0.0522, 0.0833],
        [0.0660, 0.7443, 0.5947],
        [0.1371, 0.1337, 0.8427],
        [0.7600, 0.7110, 0.8661]])

tensor([[0.7842, 0.3612, 0.9707]])
tensor([[0.4894]])


## MNIST -NN

In [0]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

# transform the raw dataset into tensors and normalize them in a fixed range
_tasks = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]) #pass mean 0.5 and std 0.5

## Load MNIST Dataset and apply transformations
mnist = MNIST("data", download=True, train=True, transform=_tasks)

## create training and validation split 
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]

## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

## create iterator objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler)


for data, label in trainloader:
  print(np.shape(data))
  # Flatten MNIST images into a 784 long vector
  # data = data.view(data.shape[0], -1)
  # print(data.shape)

  data = torch.flatten(data, start_dim=1)
  print(data.shape)
  break

torch.Size([256, 1, 28, 28])
torch.Size([256, 784])


### NN

In [0]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

# transform the raw dataset into tensors and normalize them in a fixed range
_tasks = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

## Load MNIST Dataset and apply transformations
mnist = MNIST("data", download=True, train=True, transform=_tasks)

## create training and validation split 
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]

## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

## create iterator objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler)

## Build class of model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784, 128)
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        x = self.hidden(x)
        x = torch.sigmoid(x)
        x = self.output(x)
        return x

model = Model()

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)

for epoch in range(1,11):

  train_loss, valid_loss = [], []
  model.train() # activates training mod

  ## Training on 1 epoch
  for data, target in trainloader:

    data = torch.flatten(data, start_dim=1)

    optimizer.zero_grad() #clears gradients of all optimized classes

    ## forward pass
    output = model(data)

    ## loss calc
    loss = loss_function(output, target)

    ## backeard propagation
    loss.backward()

    ## weight optimization
    optimizer.step() #performs a single optimization step
    train_loss.append(loss.item())

  ### Evaluation on 1 epoch
  for data, target in validloader:

    data = torch.flatten(data, start_dim=1)

    output = model(data)
    loss = loss_function(output, target)
    valid_loss.append(loss.item())
  
  print("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

Epoch: 1 Training Loss:  1.4278296362212364 Valid Loss:  0.7039066375570094
Epoch: 2 Training Loss:  0.5753547265491588 Valid Loss:  0.446183591446978
Epoch: 3 Training Loss:  0.43230179998468843 Valid Loss:  0.37309577490421053
Epoch: 4 Training Loss:  0.3773977352266616 Valid Loss:  0.3384805096590773
Epoch: 5 Training Loss:  0.34632741073344614 Valid Loss:  0.3146806723893957
Epoch: 6 Training Loss:  0.3253439431019286 Valid Loss:  0.299487607593232
Epoch: 7 Training Loss:  0.3091939938987823 Valid Loss:  0.28539510832187975
Epoch: 8 Training Loss:  0.2957918224658104 Valid Loss:  0.27507571083434085
Epoch: 9 Training Loss:  0.28438072516880136 Valid Loss:  0.2652054229315291
Epoch: 10 Training Loss:  0.27375956244291144 Valid Loss:  0.2566439474516727


### Validation NN

In [0]:
## dataloader for validation dataset 
dataiter = iter(validloader)
data, labels = dataiter.next()
data = torch.flatten(data, start_dim=1)
output = model(data)

print(output.shape)
print(output[0])

_, pred_tensor = torch.max(output, 1)

print(pred_tensor.shape)
print(pred_tensor[0])

preds = np.squeeze(pred_tensor.numpy())

print("Actual: ", labels[:10])
print("Predic: ", preds[:10])

torch.Size([256, 10])
tensor([-3.2789, -1.0494,  0.6679,  0.9565, -1.4610,  1.6791, -3.1327, -2.5092,
         8.5172,  0.7382], grad_fn=<SelectBackward>)
torch.Size([256])
tensor(8)
Actual:  tensor([8, 8, 7, 9, 6, 8, 3, 7, 7, 5])
Predic:  [8 5 7 9 6 8 8 7 7 5]


## MNIST - NN [1 GPU]

In [0]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from torch.backends import cudnn

cudnn.benchmark = True

# transform the raw dataset into tensors and normalize them in a fixed range
_tasks = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

## Load MNIST Dataset and apply transformations
mnist = MNIST("data", download=True, train=True, transform=_tasks)

## create training and validation split 
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]

## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

## create iterator objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler, num_workers=2)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler, num_workers=2)

## GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Build class of model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784, 128)
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        x = self.hidden(x)
        x = torch.sigmoid(x)
        x = self.output(x)
        return x

model = Model()

model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)

for epoch in range(1,11):

  train_loss, valid_loss = [], []
  model.train() # activates training mod

  ## Training on 1 epoch
  for data, target in trainloader:

    data = torch.flatten(data.to(device), start_dim=1)

    optimizer.zero_grad() #clears gradients of all optimized classes

    ## forward pass
    output = model(data.to(device))

    ## loss calc
    loss = loss_function(output.to(device), target.to(device))

    ## backeard propagation
    loss.backward()

    ## weight optimization
    optimizer.step() #performs a single optimization step
    train_loss.append(loss.item())

  ### Evaluation on 1 epoch
  for data, target in validloader:

    data = torch.flatten(data, start_dim=1)

    output = model(data.to(device))
    loss = loss_function(output.to(device), target.to(device))
    valid_loss.append(loss.item())
  
  print("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

cuda:0


## MNIST - NN [Multy GPU - Multy Core]

Specify certain GPUs

In [36]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2" # number of gpu devices
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from torch.backends import cudnn
import multiprocessing

cudnn.benchmark = True

n_cores = multiprocessing.cpu_count()

# transform the raw dataset into tensors and normalize them in a fixed range
_tasks = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

## Load MNIST Dataset and apply transformations
mnist = MNIST("data", download=True, train=True, transform=_tasks)

## create training and validation split 
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]

## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

## create iterator objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler, num_workers=n_cores)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler, num_workers=n_cores)

## GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Build class of model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784, 128)
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        x = self.hidden(x)
        x = torch.sigmoid(x)
        x = self.output(x)
        return x

model = Model()

## Multi GPU
if torch.cuda.device_count() > 1:
  print("We can use", torch.cuda.device_count(), "GPUs")
  model = nn.DataParallel(model, device_ids=[1]) # device_ids=[0,1,2] depending on the # of gpus

model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)

for epoch in range(1,11):

  train_loss, valid_loss = [], []
  model.train() # activates training mod

  ## Training on 1 epoch
  for data, target in trainloader:

    data = torch.flatten(data.to(device), start_dim=1)

    optimizer.zero_grad() #clears gradients of all optimized classes

    ## forward pass
    output = model(data.to(device))

    ## loss calc
    loss = loss_function(output.to(device), target.to(device))

    ## backeard propagation
    loss.backward()

    ## weight optimization
    optimizer.step() #performs a single optimization step
    train_loss.append(loss.item())

  ### Evaluation on 1 epoch
  for data, target in validloader:

    data = torch.flatten(data, start_dim=1)

    output = model(data.to(device))
    loss = loss_function(output.to(device), target.to(device))
    valid_loss.append(loss.item())
  
  print("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size

KeyboardInterrupt: ignored

## MNIST CNN [Multy GPU, CPU]

In [53]:
import os
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.backends import cudnn
import numpy as np
import multiprocessing

cudnn.benchmark = True

num_cores = multiprocessing.cpu_count()

# transform the raw dataset into tensors and normalize them in a fixed range
_tasks = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

## Load MNIST Dataset and apply transformations
mnist = MNIST("data", download=True, train=True, transform=_tasks)

## create training and validation split 
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]

## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampled = SubsetRandomSampler(valid_idx)

## create iterator objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler, num_workers=num_cores)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler, num_workers=num_cores)

## GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Build class of model
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    
    ## define layers
    self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.linear1 = nn.Linear(64*3*3, 512)
    self.linear2 = nn.Linear(512,10)
    
    return

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1,64*3*3) #torch.flatten(x, start_dim=1) ## reshaping
    x = F.relu(self.linear1(x))
    x = self.linear2(x)

    return x

## create model
model = Model()

## in case of multi gpu
if torch.cuda.device_count() > 1:
  print("Using", torch.cuda.device_count(), "GPUs")
  model = nn.DataParallel(model, device_ids=[1]) # [0,1,2,3]

## put model on gpu
model.to(device)

## loss fucntion
loss_function = nn.CrossEntropyLoss()
## optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)
## run for n epochs
for epoch in range(1,11):
  train_loss , valid_loss = [], []

  ## train part
  model.train()
  for data, target in trainloader:
    ## gradients acumulate. need to clear them on each example
    optimizer.zero_grad()
    output = model(data.to(device))
    loss = loss_function(output.to(device), target.to(device))
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

  ## evaluation part on validation
  model.eval() ##set model in evaluation mode
  for data, target in validloader:
    output = model(data.to(device))
    loss = loss_function(output.to(device), target.to(device))
    valid_loss.append(loss.item())

  print("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

Epoch: 1 Training Loss:  1.3867575062557738 Valid Loss:  0.20830699516103623
Epoch: 2 Training Loss:  0.14508407129014425 Valid Loss:  0.10438944716402825
Epoch: 3 Training Loss:  0.0902247162773571 Valid Loss:  0.07995569214541862
Epoch: 4 Training Loss:  0.06947706440622185 Valid Loss:  0.08476778730115991
Epoch: 5 Training Loss:  0.05636777369146968 Valid Loss:  0.06578631869497452
Epoch: 6 Training Loss:  0.048184677641442485 Valid Loss:  0.05531598358078206
Epoch: 7 Training Loss:  0.04294469940694089 Valid Loss:  0.05248951709809455
Epoch: 8 Training Loss:  0.038696830844546254 Valid Loss:  0.048144756558727714
Epoch: 9 Training Loss:  0.03434643990043154 Valid Loss:  0.04841103820883213


#### Evaluation